In [ ]:
pip install pmdarima

In [ ]:
import os 
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
from matplotlib import pyplot as plt

# LIBRARIES TO BE UTILIZED FOR ARIMA
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.api import ExponentialSmoothing,SimpleExpSmoothing,Holt
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
import pmdarima as pm
from pmdarima.model_selection import train_test_split
from fbprophet import Prophet
from pandas import to_datetime
import itertools
import warnings
import datetime
from datetime import datetime
from wordcloud import WordCloud
import matplotlib.ticker as ticker
import plotly

from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame



In [ ]:
pip install --upgrade plotly

In [ ]:
warnings.filterwarnings('ignore')

# LOADING THE DATA 

In [ ]:
import os
for dirname , _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname,filename))

datafolder ='../input/store-sales-time-series-forecasting/'

In [ ]:
oil= pd.read_csv(os.path.join(datafolder,'oil.csv'))
holidays_events=pd.read_csv(os.path.join(datafolder,'holidays_events.csv'))
stores= pd.read_csv(os.path.join(datafolder,'stores.csv'))
train= pd.read_csv(os.path.join(datafolder,'train.csv'))
df_test= pd.read_csv(os.path.join(datafolder,'test.csv'))
transactions= pd.read_csv(os.path.join(datafolder,'transactions.csv'))
sample_submission=pd.read_csv(os.path.join(datafolder,'sample_submission.csv'))

In [ ]:
holidays_events.head()

In [ ]:
stores.head()

In [ ]:
df2=pd.merge(stores,train,on='store_nbr', how="right")

In [ ]:
df2

In [ ]:
stores_transactions=pd.merge(stores,transactions,on='store_nbr',how='left')

In [ ]:
#Checking the number of unique values
stores_transactions["store_nbr"].nunique()

In [ ]:
Train=train

In [ ]:
# Investigate all the elements within each feature

for column in Train :
    unique_vals = np.unique(Train[column])
    nr_values = len(unique_vals)
    if nr_values < 30 :
        print('The number of values for feature{} :{}--{}'.format(column,nr_values,unique_vals))
    else :
            print('The number of Values for feature {}:{}'.format(column,nr_values))

In [ ]:
#Checking the number of unique values
stores_transactions["date"].value_counts()

In [ ]:
stores_transactions.isnull().sum()

In [ ]:
#Checking the number of unique values
stores_transactions["cluster"].value_counts().plot(kind='bar',figsize=(15,5))

### It seem as though most stores fall under Cluster 3
### with the least store number categorized as Cluster 5

In [ ]:
stores_transactions.columns.values

In [ ]:
#Taking the sum of all transactions over the dates

date_group = stores_transactions.groupby('date')["transactions"].sum().reset_index()

In [ ]:
date_group.head()

In [ ]:
stores_transactions.head()

# Transactions per city

In [ ]:
city_transactions= stores_transactions.groupby(['city','type'])["transactions"].sum().reset_index()

In [ ]:
city_transactions.head()

In [ ]:
plt.figure(figsize=(14, 5))
sns.barplot(x="transactions", y="city",  data=city_transactions)

### In terms of city transactions, Quito takes it by a margin.
### Puyo seems to be experiencing the least amount of transactional volumes.


# Transactions per Cluster

In [ ]:
cluster_trans= stores_transactions.groupby(['cluster','type'])["transactions"].sum().reset_index()

In [ ]:
cluster_trans.head()

In [ ]:
plt.figure(figsize=(14, 5))
sns.barplot(x="cluster", y="transactions",  data=cluster_trans)

### In terms of number of transaction per cluster, Cluster 14 is the leading one followed by  Cluster 6, 8 and 11 respectively
### The one with the least transactions is cluster 16 floowed by 12 and 7 respectively.

In [ ]:
cluster_trans= stores_transactions.groupby(['cluster','type'])["transactions"].sum().reset_index()

# Lineplot of the entire data transactions

In [ ]:


plt.figure(figsize=(16, 6))

ax=sns.lineplot(y="transactions",x="date",data=date_group)
ax.xaxis.set_major_locator(ticker.LinearLocator(10))

# Sales Per Cluster

In [ ]:
cluster_sales= df2.groupby(['cluster','type'])["sales"].sum().reset_index()

plt.figure(figsize=(14, 5))
sns.barplot(x="cluster", y="sales",hue='type' , data=cluster_sales)

In [ ]:
cluster_sales.head()

### Cluster 14 has the most/highest sales followed by Cluster 6 and 8 respectively.
### The one with the least sales being cluster number 9 and 16 respectively , with 9 being the lowest perfomer amongst the 17 clusters.
### The most common Transaction Type is D followed by A and C, with E being the least
### Cluster 10 and 6 seems to experience more than one type of transactions.

# Average sales Per city by type

In [ ]:
city_sales=df2.groupby(['date','city','type'])["sales"].mean().reset_index()
plt.figure(figsize=(16, 6))
sns.barplot(y="sales",x="type",hue="city", data=city_sales)

### It seems that Quito has the most average sales compared to other cities.
### The type with the highest average sales is A followed by D respectively
### Type A is also the least occuring trasanction Type amongst the Cities